In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import mysql.connector
from datetime import datetime, date
from tkcalendar import DateEntry

class FlightReservationSystem(tk.Tk):
    def __init__(self):
        super().__init__()

        self.title("Flight Reservation System")
        self.geometry("800x600")

        # Connect to the MySQL database
        self.db_connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="Nikhil@1305",
            database="flight_reservation"
        )
        self.db_cursor = self.db_connection.cursor()

        self.current_user = None

        self.create_widgets()

    def create_widgets(self):
        self.create_navbar()
        self.create_user_login_screen()

    def create_navbar(self):
        self.navbar = tk.Frame(self, bd=2, relief=tk.RAISED)
        self.navbar.pack(side=tk.TOP, fill=tk.X)

        home_btn = tk.Button(self.navbar, text="Home", command=self.check_login(self.create_main_screen))
        home_btn.pack(side=tk.LEFT)

        search_btn = tk.Button(self.navbar, text="Book your Flight", command=self.check_login(self.create_search_flights_screen))
        search_btn.pack(side=tk.LEFT)

        bookings_btn = tk.Button(self.navbar, text="My Bookings", command=self.check_login(self.create_check_bookings_screen))
        bookings_btn.pack(side=tk.LEFT)

        contact_btn = tk.Button(self.navbar, text="Contact Us", command=self.check_login(self.create_contact_us_screen))
        contact_btn.pack(side=tk.LEFT)

        help_btn = tk.Button(self.navbar, text="Help", command=self.check_login(self.create_help_screen))
        help_btn.pack(side=tk.LEFT)

        user_btn = tk.Button(self.navbar, text="User Account", command=self.create_user_account_screen)
        user_btn.pack(side=tk.RIGHT)

    def check_login(self, func):
        def wrapper():
            if self.current_user:
                func()
            else:
                messagebox.showerror("Login Required", "Please log in first.")
                self.create_user_login_screen()
        return wrapper

    def clear_screen(self):
        for widget in self.winfo_children():
            if widget != self.navbar:
                widget.destroy()

    def create_main_screen(self):
        self.clear_screen()
        main_frame = tk.Frame(self)
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        main_content_frame = tk.Frame(main_frame)
        main_content_frame.place(relx=0.5, rely=0.5, anchor=tk.CENTER)

        title = tk.Label(main_content_frame, text="Welcome to the Flight Reservation System", font=("Helvetica", 18))
        title.pack(pady=20)

      
    def create_search_flights_screen(self):
        self.clear_screen()
        search_frame = tk.Frame(self)
        search_frame.pack(fill=tk.BOTH, expand=True)

        airports_in_india = [
            "Delhi (DEL)",
            "Mumbai (BOM)",
            "Bengaluru (BLR)",
            "Kolkata (CCU)",
            "Hyderabad (HYD)",
            "Chennai (MAA)",
            "Ahmedabad (AMD)",
            "Kochi (COK)",
            "Goa (GOI)",
            "Pune (PNQ)",
            "Jaipur (JAI)",
            "Thiruvananthapuram (TRV)",
            "Bhubaneswar (BBI)",
            "Guwahati (GAU)",
            "Lucknow (LKO)",
            "Amritsar (ATQ)",
            "Indore (IDR)",
            "Nagpur (NAG)",
            "Ranchi (IXR)",
            "Port Blair (IXZ)",
            "Varanasi (VNS)",
            "Visakhapatnam (VTZ)",
            "Siliguri (IXB)",
            "Surat (STV)",
            "Madurai (IXM)",
            "Tiruchirappalli (TRZ)",
            "Vadodara (BDQ)",
            "Mangalore (IXE)",
            "Patna (PAT)",
            "Shillong (SHL)"
        ]

        tk.Label(search_frame, text="Departure City").grid(row=0, column=0, padx=10, pady=10)
        self.departure_city_combo = ttk.Combobox(search_frame, values=airports_in_india)
        self.departure_city_combo.grid(row=0, column=1, padx=0, pady=10)

        tk.Label(search_frame, text="Destination City").grid(row=1, column=0, padx=10, pady=10)
        self.destination_city_combo = ttk.Combobox(search_frame, values=airports_in_india)
        self.destination_city_combo.grid(row=1, column=1, padx=10, pady=10)

        tk.Label(search_frame, text="Departure Date").grid(row=2, column=0, padx=10, pady=10)
        self.departure_date_entry = DateEntry(search_frame, width=12, background='darkblue',foreground='white', borderwidth=2, date_pattern='yyyy-mm-dd')
        self.departure_date_entry.grid(row=2, column=1, padx=10, pady=10)

        tk.Label(search_frame, text="Return Date").grid(row=3, column=0, padx=10, pady=10)
        self.return_date_entry = DateEntry(search_frame, width=12, background='darkblue',foreground='white', borderwidth=2, date_pattern='yyyy-mm-dd')
        self.return_date_entry.grid(row=3, column=1, padx=10, pady=10)

        tk.Label(search_frame, text="Number of Passengers").grid(row=4, column=0, padx=10, pady=10)
        self.passengers_entry = tk.Entry(search_frame)
        self.passengers_entry.grid(row=4, column=1, padx=10, pady=10)

        tk.Button(search_frame, text="Submit", command=self.store_flight_data).grid(row=5, column=0, columnspan=2, pady=20)

    def validate_dates(self):
        departure_date = self.departure_date_entry.get_date()
        return_date = self.return_date_entry.get_date()
        today = date.today()

        if departure_date < today:
            messagebox.showerror("Invalid Date", "Departure date cannot be in the past.")
            return False
        if return_date < today:
            messagebox.showerror("Invalid Date", "Return date cannot be in the past.")
            return False
        if return_date < departure_date:
            messagebox.showerror("Invalid Date", "Return date cannot be before the departure date.")
            return False
        return True

    def store_flight_data(self):
        if not self.validate_dates():
            return
        departure_city = self.departure_city_combo.get()
        destination_city = self.destination_city_combo.get()
        departure_date = self.departure_date_entry.get_date()
        return_date = self.return_date_entry.get_date()
        passengers = self.passengers_entry.get()

        if not (departure_city and destination_city and departure_date and return_date and passengers):
            messagebox.showerror("Input Error", "All fields are required.")
            return

        try:
            passengers = int(passengers)
        except ValueError:
            messagebox.showerror("Input Error", "Number of Passengers must be an integer.")
            return

        query = "INSERT INTO flights (departure_city, destination_city, departure_date, return_date, passengers) " \
                "VALUES (%s, %s, %s, %s, %s)"
        flight_data = (departure_city, destination_city, departure_date, return_date, passengers)
        
        try:
            self.db_cursor.execute(query, flight_data)
            self.db_connection.commit()
            
            # Retrieve the ID of the inserted record
            flight_id = self.db_cursor.lastrowid
            messagebox.showinfo("Success", f"Flight data stored successfully.\nBooking ID: {flight_id}")
        except mysql.connector.Error as err:
            messagebox.showerror("Error", f"Error storing flight data: {err}")


    def create_search_results_screen(self):
        self.clear_screen()
        results_frame = tk.Frame(self)
        results_frame.pack(fill=tk.BOTH, expand=True)

        tk.Label(results_frame, text="Search Results", font=("Helvetica", 16)).pack(pady=10)
        
        # Placeholder for search results
        result = tk.Label(results_frame, text="(Flight results will be displayed here)")
        result.pack()

    def create_check_bookings_screen(self):
        self.clear_screen()
        bookings_frame = tk.Frame(self)
        bookings_frame.pack(fill=tk.BOTH, expand=True)

        tk.Label(bookings_frame, text="Check My Bookings", font=("Helvetica", 16)).pack(pady=10)
        
        tk.Label(bookings_frame, text="Enter Booking ID").pack(pady=5)
        self.booking_id_entry = tk.Entry(bookings_frame)
        self.booking_id_entry.pack(pady=5)

        tk.Button(bookings_frame, text="Check", command=self.check_booking).pack(pady=10)

    def check_booking(self):
        booking_id = self.booking_id_entry.get()

        if not booking_id:
            messagebox.showerror("Input Error", "Booking ID is required.")
            return

        try:
            booking_id = int(booking_id)
        except ValueError:
            messagebox.showerror("Input Error", "Booking ID must be an integer.")
            return

        query = "SELECT * FROM flights WHERE id = %s"
        self.db_cursor.execute(query, (booking_id,))
        booking = self.db_cursor.fetchone()

        if booking:
            self.show_booking_info(booking)
        else:
            messagebox.showinfo("No Booking Found", f"No booking found with ID {booking_id}.")

    def show_booking_info(self, booking):
        self.clear_screen()
        info_frame = tk.Frame(self)
        info_frame.pack(fill=tk.BOTH, expand=True)

        tk.Label(info_frame, text="Booking Information", font=("Helvetica", 16)).pack(pady=10)

        info = f"""
        Booking ID: {booking[0]}
        Departure City: {booking[1]}
        Destination City: {booking[2]}
        Departure Date: {booking[3]}
        Return Date: {booking[4]}
        Number of Passengers: {booking[5]}
        """
        tk.Label(info_frame, text=info, justify=tk.LEFT).pack(pady=10)


    def create_contact_us_screen(self):
        self.clear_screen()
        contact_frame = tk.Frame(self)
        contact_frame.pack(fill=tk.BOTH, expand=True)

        tk.Label(contact_frame, text="Contact Us", font=("Helvetica", 16)).pack(pady=10)
        
        tk.Label(contact_frame, text="Name").pack(pady=5)
        self.contact_name_entry = tk.Entry(contact_frame)
        self.contact_name_entry.pack(pady=5)

        tk.Label(contact_frame, text="Email").pack(pady=5)
        self.contact_email_entry = tk.Entry(contact_frame)
        self.contact_email_entry.pack(pady=5)

        tk.Label(contact_frame, text="Message").pack(pady=5)
        self.contact_message_entry = tk.Text(contact_frame, height=5)
        self.contact_message_entry.pack(pady=5)

        tk.Button(contact_frame, text="Submit", command=self.submit_contact_form).pack(pady=10)

    def submit_contact_form(self):
        name = self.contact_name_entry.get()
        email = self.contact_email_entry.get()
        message = self.contact_message_entry.get("1.0", tk.END).strip()

        if not (name and email and message):
            messagebox.showerror("Input Error", "All fields are required.")
            return

        query = "INSERT INTO contacts (name, email, message) VALUES (%s, %s, %s)"
        values = (name, email, message)
        
        self.db_cursor.execute(query, values)
        self.db_connection.commit()

        messagebox.showinfo("Success", "Message sent successfully.")
        self.create_contact_us_screen()

    def create_help_screen(self):
        self.clear_screen()
        help_frame = tk.Frame(self)
        help_frame.pack(fill=tk.BOTH, expand=True)

        tk.Label(help_frame, text="Help", font=("Helvetica", 16)).pack(pady=10)
        
        help_text = """
        For assistance, please contact our support team call 1-800-FLY-NOW.
        
        FAQs:
        1. How do I book a flight?
        - Navigate to 'Book your Flights' and enter your travel details to book flights.

        2. How do I check my bookings?
        - Go to 'My Bookings' and enter your booking ID to view your booking details.

        3. How do I contact support?
        - Use the 'Contact Us' form to send us a message or call our support number.
        """
        
        tk.Label(help_frame, text=help_text, justify=tk.LEFT).pack(pady=10)

    def create_user_login_screen(self):
        self.clear_screen()
        login_frame = tk.Frame(self)
        login_frame.pack(fill=tk.BOTH, expand=True)

        tk.Label(login_frame, text="User Login", font=("Helvetica", 16)).pack(pady=10)

        tk.Label(login_frame, text="Username").pack(pady=5)
        self.login_username_entry = tk.Entry(login_frame)
        self.login_username_entry.pack(pady=5)

        tk.Label(login_frame, text="Password").pack(pady=5)
        self.login_password_entry = tk.Entry(login_frame, show="*")
        self.login_password_entry.pack(pady=5)

        tk.Button(login_frame, text="Login", command=self.user_login).pack(pady=10)

        tk.Button(login_frame, text="Register", command=self.create_user_registration_screen).pack(pady=10)

    def user_login(self):
        username = self.login_username_entry.get()
        password = self.login_password_entry.get()

        if not (username and password):
            messagebox.showerror("Input Error", "Username and Password are required.")
            return

        query = "SELECT * FROM users WHERE username = %s AND password = %s"
        self.db_cursor.execute(query, (username, password))
        user = self.db_cursor.fetchone()

        if user:
            self.current_user = user
            messagebox.showinfo("Success", f"Welcome {username}!")
            self.create_main_screen()
        else:
            messagebox.showerror("Login Error", "Invalid username or password.")
    
    def create_user_registration_screen(self):
        self.clear_screen()
        register_frame = tk.Frame(self)
        register_frame.pack(fill=tk.BOTH, expand=True)

        tk.Label(register_frame, text="User Registration", font=("Helvetica", 16)).pack(pady=10)

        tk.Label(register_frame, text="Username").pack(pady=5)
        self.register_username_entry = tk.Entry(register_frame)
        self.register_username_entry.pack(pady=5)

        tk.Label(register_frame, text="Password").pack(pady=5)
        self.register_password_entry = tk.Entry(register_frame, show="*")
        self.register_password_entry.pack(pady=5)

        tk.Label(register_frame, text="Email").pack(pady=5)
        self.register_email_entry = tk.Entry(register_frame)
        self.register_email_entry.pack(pady=5)

        tk.Button(register_frame, text="Register", command=self.register_user).pack(pady=10)

    def register_user(self):
        username = self.register_username_entry.get()
        password = self.register_password_entry.get()
        email = self.register_email_entry.get()

        if not (username and password and email):
            messagebox.showerror("Input Error", "All fields are required.")
            return

        query = "SELECT * FROM users WHERE username = %s"
        self.db_cursor.execute(query, (username,))
        existing_user = self.db_cursor.fetchone()

        if existing_user:
            messagebox.showerror("Registration Error", "Username already exists.")
            return

        query = "INSERT INTO users (username, password, email) VALUES (%s, %s, %s)"
        values = (username, password, email)
        
        self.db_cursor.execute(query, values)
        self.db_connection.commit()

        messagebox.showinfo("Success", "Registration successful. Please log in.")
        self.create_user_login_screen()

    def create_user_account_screen(self):
        if not self.current_user:
            self.create_user_login_screen()
            return

        self.clear_screen()
        account_frame = tk.Frame(self)
        account_frame.pack(fill=tk.BOTH, expand=True)

        tk.Label(account_frame, text="User Account", font=("Helvetica", 16)).pack(pady=10)

        username = self.current_user[1]  # Assuming the second column is the username
        email = self.current_user[3]  # Assuming the fourth column is the email

        account_info = f"""
        Username: {username}
        Email: {email}
        """
        tk.Label(account_frame, text=account_info, justify=tk.LEFT).pack(pady=10)

        tk.Button(account_frame, text="Logout", command=self.user_logout).pack(pady=10)

    def user_logout(self):
        self.current_user = None
        messagebox.showinfo("Logged Out", "You have been logged out.")
        self.create_user_login_screen()

if __name__ == "__main__":
    app = FlightReservationSystem()
    app.mainloop()